In [851]:
import numpy as np
import pandas as pd
import glob
import sys,os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from collections import defaultdict
import pymp
import qgrid
import plotly.express as px
import gezi
from gezi import tqdm, line
# tqdm.pandas()
from IPython.display import display
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [852]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [853]:
STEP = 3
FOLDS = 5
def gen_metrics(root, folds=5, step=STEP, force_full=False):
  dfs= []
  fold_counts = defaultdict(int)
  for fold in range(folds):
    pattern = f'{root}/{fold}/*/metrics.csv'
    files = glob.glob(pattern)
    if not files:
      continue
    files = sorted(files, key=lambda x: os.path.getmtime(x))
    for file in files:
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      if not len(df):
        continue
      if force_full and len(df) != step:
        continue
      if 'step' not in df.columns:
        df['step'] = range(len(df))
      model = os.path.basename(os.path.dirname(file))
      try:
        if df.step.max() >= step:
          fold_counts[model] += 1
      except Exception:
        # ic(df)
        continue
      df['model'] = model
      df['mtime'] = os.path.getmtime(file)
      df['ctime'] = os.path.getctime(file)
      df['fold'] = fold
      dfs.append(df)
      # dfs = sorted(dfs, key=lambda x: -len(x.columns))
  
  df = pd.concat(list(dfs))
  df['num_folds'] = df.model.apply(lambda x: fold_counts[x])
  return df

In [854]:
def show(metric, silent=False, num=100, step=STEP, folds=FOLDS):
  d = df
  d = d[d.step == step]
  # d = d.sort_values('Overall', ascending=False).drop_duplicates(['model', 'fold'])
  # for fold in range(folds):
  #   print('fold:', fold)
  #   d_ = d[d.fold==fold].sort_values([metric], ascending=False).reset_index(drop=True)
  #   if not len(d_):
  #     continue
  #   d_[f'{metric}_str'] = ['%.4f' % x for x in d_[metric].values]
  #   display(px.bar(d_, x=metric, y='model',  color='model', text=f'{metric}_str'))
  # if not silent:
 
  fig = line(d, metric, x='fold', color='model', return_figs=True)[0]
  if not silent:
    print(f'{metric}: per fold')
    display(fig)
    # print('f1 mean')
    # fig = line(d, metric, x='fold', color='model', mean_smooth=True, return_figs=True)[0]
    # display(fig)
  start = 0 if not silent else folds - 1
  for fold in range(start, folds):
    print(f'{metric}: num_folds', fold + 1)
    d_ = d
    d = d[d.num_folds >= fold + 1]
    d = d[d.fold <= fold]
    if not len(d):
      continue
    d = d.groupby(['model'])[metric] \
      .aggregate(np.mean).reset_index()
    d = d[['model', metric]].sort_values([metric], ascending=False).reset_index(drop=True)
    d = d.head(num)
    d[f'{metric}_str'] = ['%.4f' % x for x in d[metric].values]
    display(px.bar(d, x=metric, y='model',  color='model', text=f'{metric}_str'))
    d = d_

In [855]:
v = '60'
# type = 'tiny'
# type = 'mid'
# type = 'large'
mark = 'offline'
# root = f'../working/{mark}/{v}.*'
root = f'../working/{mark}/{v}'
classes = [
  'Claim', 'Evidence', 'Position', 'Concluding Statement', 'Lead', 'Counterclaim', 'Rebuttal'
]

In [856]:
try:
  df = gen_metrics(root)
except Exception:
  df = gen_metrics(root, force_full=True)

In [857]:
models = set(df.model)
# mark = 'longformer'
# models = [x for x in models if mark in x]
# mark = '.len'
# models = [x for x in models if mark not in x]
df = df[df.model.isin(models)]
df = df[df['f1/Overall'] > 0.5]

In [858]:
models = list(set(df[df['f1/Overall'] < 0.5].model))
models

[]

In [859]:
df[df['f1/Overall'] < 0.5][['model', 'fold', 'f1/Overall']].drop_duplicates()

,model,fold,f1/Overall


In [860]:
 list(set(df[df['f1/Overall'] > 0.65].model))

['bird.start.len1024.bird_ori',
 'deberta-xlarge.start.stride-256',
 'bart.start.stride-256',
 'bart.se',
 'deberta-v3.se.len1024',
 'deberta-v3.start.len1024.rnn_stack',
 'deberta.se',
 'deberta-v3.start.stride-256',
 'deberta-v3.start.len1024.stride-512',
 'deberta-xlarge.start',
 'deberta-v3.start.len1536',
 'deberta-v3.se2',
 'deberta-v3.end.len1024',
 'tiny.start.len1024.stride-256',
 'deberta-v3.start.len1024.stride-512.seq_encoder-0',
 'deberta-v3.start.len1024.stride-256',
 'deberta-v3.start.len1024.rnn_bi',
 'deberta-v3.se',
 'tiny.start.len1024',
 'deberta-v3.mid.len1024',
 'deberta-v3.start.mui-end-mid',
 'deberta-v3.end.len1024.seq_encoder-0',
 'bird.start.len1024',
 'deberta-v3.start.stride-128',
 'deberta-v3.start.len1024',
 'longformer.start.len1536',
 'deberta.start.mui-end-mid',
 'deberta.start.stride-128',
 'deberta-v3.start.len1024.ucm',
 'deberta-v3.start.len1024.stride-256.seq_encoder-0',
 'bart.start.stride-128',
 'deberta-v3.se2.len1024',
 'tiny.start.len1024.str

In [861]:
show('f1/Overall')

f1/Overall: per fold


f1/Overall: num_folds 1


f1/Overall: num_folds 2


f1/Overall: num_folds 3


f1/Overall: num_folds 4


f1/Overall: num_folds 5


In [862]:
# show('f1_1024+/Overall')

In [863]:
show('f1_400-/Overall', False)

f1_400-/Overall: per fold


f1_400-/Overall: num_folds 1


f1_400-/Overall: num_folds 2


f1_400-/Overall: num_folds 3


f1_400-/Overall: num_folds 4


f1_400-/Overall: num_folds 5


In [864]:
show('auc/sep', True)

auc/sep: num_folds 5


In [865]:
show('f1/Binary', True)

f1/Binary: num_folds 5


In [866]:
show('auc/token', True)

auc/token: num_folds 5


In [867]:
for cls_ in classes:
  show(f'f1/{cls_}', silent=True)

f1/Claim: num_folds 5


f1/Evidence: num_folds 5


f1/Position: num_folds 5


f1/Concluding Statement: num_folds 5


f1/Lead: num_folds 5


f1/Counterclaim: num_folds 5


f1/Rebuttal: num_folds 5


In [868]:
show('f1/Binary', True)

f1/Binary: num_folds 5


In [869]:
d = pd.read_csv(f'../working/offline/{v}/ensemble.csv')
d = d.sort_values(['f1/Overall'], ascending=False)

In [870]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
# d[['f1/Overall', 'F1/Overall', 'mns']]
d[['f1/Overall','mns']]

,f1/Overall,mns
63,0.720775,15_deberta-v3.start|deberta-v3.end|deberta-v3.se|deberta-v3.start.stride-256|deberta.start|deberta.end|deberta.start.stride-256|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536
56,0.720775,15_deberta-v3.start|deberta-v3.start.stride-256|deberta-v3.end|deberta-v3.se|deberta.start|deberta.start.stride-256|deberta.end|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536
60,0.720775,15_deberta-v3.start|deberta-v3.end|deberta-v3.se|deberta-v3.start.stride-256|deberta.start|deberta.end|deberta.start.stride-256|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536
55,0.720591,14_deberta-v3.start|deberta-v3.start.stride-256|deberta-v3.end|deberta-v3.se|deberta.start.stride-256|deberta.end|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536
57,0.720464,14_deberta-v3.start|deberta-v3.start.stride-256|deberta-v3.end|deberta-v3.se|deberta.start|deberta.start.stride-256|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536
53,0.720453,14_deberta-v3.start|deberta-v3.start.stride-256|deberta-v3.end|deberta-v3.se|deberta.start|deberta.end|bart.start|longformer.start.len1536|deberta-v3.start.len1536|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0
50,0.720334,13_deberta-v3.start|deberta-v3.end|deberta-v3.se|deberta.start|deberta.end|bart.start|longformer.start.len1536|deberta-v3.start.len1536|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0
54,0.720296,12_deberta-v3.start|deberta-v3.start.stride-256|deberta.start|deberta.end|bart.start|longformer.start.len1536|deberta-v3.start.len1536|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0
62,0.720163,14_deberta-v3.start|deberta-v3.end|deberta-v3.se|deberta-v3.start.stride-256|deberta.start|deberta.end|deberta.start.stride-256|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536
61,0.720149,16_deberta-v3.start|deberta-v3.end|deberta-v3.se|deberta-v3.start.stride-256|deberta.start|deberta.end|deberta.start.stride-256|bart.start|deberta-v3.start.len1536|deberta-v3.se.len1024|deberta-v3.start.len1024.stride-512|deberta-v3.start.len1024.stride-512.seq_encoder-0|deberta-v3.start.len1024.stride-256|deberta-v3.start.len1024.stride-256.seq_encoder-0|longformer.start.len1536|bird.start.len1024


In [871]:
d[['f1/Overall','mns']].head(1)['mns'].values[0].split('|')

['15_deberta-v3.start',
 'deberta-v3.end',
 'deberta-v3.se',
 'deberta-v3.start.stride-256',
 'deberta.start',
 'deberta.end',
 'deberta.start.stride-256',
 'bart.start',
 'deberta-v3.start.len1536',
 'deberta-v3.se.len1024',
 'deberta-v3.start.len1024.stride-512',
 'deberta-v3.start.len1024.stride-512.seq_encoder-0',
 'deberta-v3.start.len1024.stride-256',
 'deberta-v3.start.len1024.stride-256.seq_encoder-0',
 'longformer.start.len1536']